# Lab Bi


In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [29]:
df = pd.read_csv('./data/data.csv', sep=';')


In [30]:
df.shape

(7471, 18)

In [31]:
df.sample(5)

,#,index,tconst,titleType,originalTitle,isAdult,startYear,runtimeMinutes,averageRating,numVotes,ordering,region,language,types,attributes,isOriginalTitle,main_genre,secondary_genre
7062,7439,9775,tt3756788,movie,Umimachi Diary,0.0,2015.0,127,7.5,15984.0,38.0,CA,en,imdbDisplay,\N,0.0,Drama,Family
6118,6483,8544,tt0442632,tvMiniSeries,Bleak House,0.0,2005.0,510,8.3,10214.0,7.0,CA,en,imdbDisplay,\N,0.0,Crime,Drama
7021,7397,9721,tt3463606,tvSeries,Welcome to Sweden,0.0,2014.0,30,6.6,5667.0,2.0,CA,en,imdbDisplay,\N,0.0,Comedy,No tiene
1994,2280,3244,tt12585152,tvSeries,Down to Earth with Zac Efron,0.0,2020.0,40,8.0,7693.0,26.0,IN,en,imdbDisplay,\N,0.0,Documentary,History
2837,3131,4297,tt1666278,tvSeries,Thundercats,0.0,2011.0,\N,7.8,7409.0,25.0,IN,en,imdbDisplay,\N,0.0,Action,Adventure
